In [13]:
import torch.nn as nn
import cv2

In [14]:
class SoftmaxClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(SoftmaxClassifier, self).__init__()
        
        # Single fully connected layer
        self.linear = nn.Linear(input_dim, num_classes)
    
    def forward(self, x):
        
        # Compute raw logits
        logits = self.linear(x)
        
        # Apply softmax to get probabilities
        probabilities = nn.softmax(logits, dim=1)
        return probabilities

In [12]:
def flatten_image(image_path):
    img_array = cv2.imread(image_path)
    flattened_array = img_array.reshape(-1)
    return flattened_array

image_path = "/Users/ericcui/repos/imagenette/imagenette2-320/train/n01440764/ILSVRC2012_val_00000293.JPEG"
flattened_array = flatten_image(image_path)

[117 144 148 ...  60  95  99]
408960


In [ ]:
input_dim = len(flattened_array)